**นายพชรพล เกตุแก้ว รหัสนักศึกษา 6610110190**
# Food Classification from Text Using CNN
- โมเดลจำแนกอาหารจากข้อความ

---

## ขั้นตอนที่ 1 นำเข้าไลบารี่ต่างๆที่จำเป็น

In [ ]:
import pandas as pd
import re
import string
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import pickle


## ขั้นตอนที่ 2 เตรียมข้อมูลข้อความสำหรับฝึกโมเดล

In [ ]:
# โหลดชุดข้อมูลและแสดงข้อมูลพื้นฐาน
df = pd.read_csv("/content/food_recommendation_dataset.csv")
print("Total samples:", len(df))
print("\nClass distribution:")
print(df['label'].value_counts())
print("\nSample data:")
print(df.head())

# ทำความสะอาดข้อความ: แปลงเป็นตัวพิมพ์เล็ก ลบอักขระที่ไม่ใช่ตัวอักษร และตัดช่องว่าง
def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.strip()
    return text

df['text'] = df['text'].apply(clean_text)
df = df[df['text'] != '']  # ลบแถวที่ข้อความว่างหลังทำความสะอาด

# แยกตัวแปรอิสระ (ข้อความ) และตัวแปรเป้าหมาย (ป้ายกำกับ)
X = df['text'].values
y = df['label'].values

# แบ่งข้อมูลเป็นชุดฝึก (80%) และชุดตรวจสอบ (20%) โดยคงสัดส่วนแต่ละคลาส
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

Total samples: 4575

Class distribution:
label
sushi        915
hamburger    915
pizza        915
ice_cream    915
apple_pie    915
Name: count, dtype: int64

Sample data:
                                                text      label
0  I want small, delicate bites with bold umami f...      sushi
1  I want a sandwich with a juicy patty between t...  hamburger
2  I want a sandwich with a juicy patty between t...  hamburger
3  I want layers of bread, meat, veggies, and con...  hamburger
4  Honestly, cheesy, saucy, and satisfying comfor...      pizza


ในส่วนของขั้นตอนนี้ทำหน้าที่  
- โหลดชุดข้อมูลจากไฟล์ CSV และแสดงจำนวนตัวอย่าง การกระจายของแต่ละหมวดอาหาร และตัวอย่างข้อมูลดิบ  
- ทำความสะอาดข้อความโดยแปลงเป็นตัวพิมพ์เล็ก ลบเครื่องหมายวรรคตอน/ตัวเลข และตัดช่องว่างส่วนเกิน เพื่อให้ข้อมูลพร้อมสำหรับการประมวลผลด้วยโมเดล NLP  
- ลบแถวที่กลายเป็นข้อความว่างหลังการทำความสะอาด เพื่อหลีกเลี่ยงข้อมูลเสีย  
- แบ่งข้อมูลออกเป็นชุดฝึกและชุดตรวจสอบโดยใช้ `train_test_split` แบบ stratified เพื่อรักษาสัดส่วนของแต่ละคลาสให้เท่ากันทั้งสองชุด ซึ่งช่วยให้การประเมินโมเดลมีความน่าเชื่อถือ

## ขั้นตอนที่ 3 แปลงข้อความและป้ายกำกับให้อยู่ในรูปแบบที่โมเดลเข้าใจ

In [ ]:
# ตั้งค่าพารามิเตอร์การแปลงข้อความเป็นตัวเลข
MAX_WORDS = 10000
MAX_LEN = 50

# สร้างและฝึกตัวตัดคำ (tokenizer) จากชุดฝึก
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train)

# แปลงข้อความเป็นลำดับตัวเลข
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

# ปรับความยาวของทุกลำดับให้เท่ากันด้วย padding
X_train_pad = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post')
X_val_pad = pad_sequences(X_val_seq, maxlen=MAX_LEN, padding='post')

# แปลงป้ายกำกับข้อความให้เป็นตัวเลข
label_encoder = LabelEncoder()
y_train_enc = label_encoder.fit_transform(y_train)
y_val_enc = label_encoder.transform(y_val)

# แปลงป้ายกำกับตัวเลขให้เป็น one-hot encoding
num_classes = len(label_encoder.classes_)
y_train_cat = to_categorical(y_train_enc, num_classes)
y_val_cat = to_categorical(y_val_enc, num_classes)

ในส่วนของขั้นตอนนี้ทำหน้าที่  
- สร้างตัวตัดคำ (Tokenizer) ที่จำกัดคำศัพท์สูงสุด 10,000 คำ และกำหนดโทเค็นพิเศษ `<OOV>` สำหรับคำที่ไม่อยู่ในพจนานุกรม  
- แปลงข้อความในชุดฝึกและชุดตรวจสอบให้เป็นลำดับตัวเลข (sequences) ตามพจนานุกรมที่ได้จากชุดฝึก  
- ปรับความยาวของทุกลำดับให้เท่ากัน (50 โทเค็น) โดยเติมศูนย์ด้านหลัง (post-padding) เพื่อให้เหมาะกับการป้อนเข้าโมเดล  
- แปลงป้ายกำกับประเภทอาหาร (เช่น 'pizza', 'sushi') ให้เป็นตัวเลขด้วย `LabelEncoder`  
- แปลงป้ายกำกับตัวเลขให้เป็นรูปแบบ one-hot encoding เพื่อใช้ในการฝึกโมเดลจำแนกหลายคลาสด้วย categorical crossentropy loss

## ขั้นตอนที่ 4 สร้างและฝึกโมเดล

In [ ]:
# กำหนดพารามิเตอร์และสร้างโมเดล
VOCAB_SIZE = min(MAX_WORDS, len(tokenizer.word_index) + 1)
EMBEDDING_DIM = 100

model = Sequential([
    Embedding(VOCAB_SIZE, EMBEDDING_DIM, input_length=MAX_LEN),
    Conv1D(128, 5, activation='relu', kernel_regularizer=l2(0.001)),
    GlobalMaxPooling1D(),
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# คอมไพล์โมเดลด้วย optimizer, loss function และ metric ที่เหมาะสม
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# แสดงโครงสร้างโมเดล
model.summary()

# ตั้งค่า early stopping เพื่อหยุดการฝึกเมื่อไม่ปรับปรุง
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

# ฝึกโมเดล
history = model.fit(
    X_train_pad, y_train_cat,
    batch_size=32,
    epochs=30,
    validation_data=(X_val_pad, y_val_cat),
    callbacks=[early_stop],
    verbose=1
)

NameError: name 'MAX_WORDS' is not defined

ในส่วนของขั้นตอนนี้ทำหน้าที่  
- สร้างโมเดล Sequential แบบ 1D-CNN สำหรับงานจำแนกข้อความ โดยเริ่มจากชั้น Embedding เพื่อแปลงคำเป็นเวกเตอร์ความหมาย  
- ใช้ชั้น Conv1D พร้อม L2 regularization เพื่อดึงคุณลักษณะสำคัญจากลำดับคำ ตามด้วย GlobalMaxPooling1D เพื่อลดมิติ  
- เพิ่มชั้น Dense พร้อม regularization และ Dropout เพื่อลด overfitting และเพิ่มความสามารถทั่วไปของโมเดล  
- คอมไพล์โมเดลด้วย optimizer ‘adam’ และ loss function ‘categorical_crossentropy’ ซึ่งเหมาะสมกับงานจำแนกหลายคลาส  
- ฝึกโมเดลด้วย batch size 32 เป็นเวลาสูงสุด 30 epochs โดยใช้ early stopping ที่ตรวจสอบความแม่นยำบนชุดตรวจสอบ และคืนน้ำหนักที่ดีที่สุดเมื่อหยุดฝึก  
- บันทึกประวัติการฝึก (history) เพื่อใช้วิเคราะห์ประสิทธิภาพโมเดลภายหลัง

## ขั้นตอนที่ 5 ประเมินผลและตรวจสอบเป้าหมายความแม่นยำของโมเดล

In [ ]:
# ประเมินผลโมเดลบนชุดตรวจสอบและตรวจสอบเป้าหมายความแม่นยำ
val_loss, val_acc = model.evaluate(X_val_pad, y_val_cat, verbose=0)
print(f"\nValidation Accuracy: {val_acc:.4f} ({val_acc*100:.2f}%)")

# ตรวจสอบว่าโมเดลบรรลุเป้าหมายความแม่นยำ >80% หรือไม่
if val_acc >= 0.80:
    print("เป้าหมาย >80% สำเร็จ!")
else:
    print("ยังไม่ถึง 80% — อาจต้องปรับ hyperparameter หรือเพิ่มข้อมูล")


✅ Validation Accuracy: 1.0000 (100.00%)
🎉 เป้าหมาย >80% สำเร็จ!


ในส่วนของขั้นตอนนี้ทำหน้าที่  
- ประเมินประสิทธิภาพของโมเดลที่ฝึกเสร็จแล้วบนชุดตรวจสอบ (validation set) โดยใช้ฟังก์ชัน `evaluate()` เพื่อคำนวณค่า loss และความแม่นยำ  
- แสดงผลลัพธ์ความแม่นยำเป็นทั้งทศนิยมและเปอร์เซ็นต์ เพื่อให้อ่านเข้าใจง่าย  
- ตรวจสอบเงื่อนไขว่าโมเดลบรรลุเป้าหมายความแม่นยำขั้นต่ำที่กำหนดไว้ (80%) หรือไม่ ซึ่งสอดคล้องกับความต้องการของระบบแนะนำเมนูที่ต้องการความแม่นยำสูง  
- ให้คำแนะนำเบื้องต้นหากยังไม่ถึงเป้าหมาย เช่น การปรับ hyperparameter หรือขยายชุดข้อมูล เพื่อช่วยในการพัฒนาโมเดลต่อไป

## ขั้นตอนที่ 6 บันทึกโมเดลและส่วนประกอบที่จำเป็นสำหรับการใช้งานจริง



In [ ]:
# บันทึกโมเดลและอ็อบเจกต์ที่จำเป็นสำหรับการ deploy หรือใช้งานในอนาคต
model.save("/content/food_classifier_model.keras")

with open('/content/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

with open('/content/label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

print("บันทึกโมเดล, tokenizer, และ label encoder เรียบร้อย!")

✅ บันทึกโมเดล, tokenizer, และ label encoder เรียบร้อย!


ในส่วนของขั้นตอนนี้ทำหน้าที่  
- บันทึกโมเดลที่ฝึกเสร็จแล้วในรูปแบบไฟล์ `.keras` ซึ่งสามารถโหลดกลับมาใช้ใหม่ได้ทันทีโดยไม่ต้องฝึกซ้ำ  
- บันทึก `tokenizer` ที่ใช้แปลงข้อความเป็นตัวเลข ลงในไฟล์ pickle เพื่อให้ระบบใหม่สามารถประมวลผลข้อความด้วยพจนานุกรมเดียวกัน  
- บันทึก `label_encoder` ที่เก็บ mapping ระหว่างชื่อหมวดอาหาร (เช่น 'pizza') กับตัวเลข index เพื่อแปลงผลลัพธ์จากโมเดลกลับเป็นชื่ออาหารที่อ่านเข้าใจได้  
- ยืนยันการบันทึกสำเร็จผ่านข้อความแสดงผล ซึ่งเป็นขั้นตอนสำคัญก่อนนำไป deploy ผ่าน FastAPI หรือใช้ในระบบแนะนำเมนูต่อไป

## ขั้นตอนที่ 7 ทดสอบการทำนายประเภทอาหารจากข้อความใหม่



In [ ]:
# ฟังก์ชันทำนายประเภทอาหารจากข้อความใหม่
def predict_food(text):
    cleaned = clean_text(text)
    seq = tokenizer.texts_to_sequences([cleaned])
    pad = pad_sequences(seq, maxlen=MAX_LEN, padding='post')
    pred = model.predict(pad, verbose=0)
    label_idx = np.argmax(pred, axis=1)[0]
    food = label_encoder.inverse_transform([label_idx])[0]
    confidence = np.max(pred)
    return food, confidence

# ตัวอย่างข้อความทดสอบเพื่อประเมินความสามารถการทำนาย
test_texts = [
    "I want something cold and sweet with chocolate sauce",
    "A food with crust, sauce, and melted cheese on top",
    "Something with pepperoni and melted cheese layers",
    "I'm craving a juicy beef patty in a soft bun",
    "Something with bread on top and bottom, meat in between",
    "Fast food that comes with fries on the side",
    "A handheld meal with cheese, lettuce, and tomato",
    "A layered dish with sauce, vegetables, and meat",
    "Something you can eat quickly with one hand",
    "I’d love something grilled and juicy between buns",
]

# ทดสอบการทำนายและแสดงผลลัพธ์พร้อมระดับความมั่นใจ
for t in test_texts:
    food, conf = predict_food(t)
    print(f"Input: '{t}' → Predicted: {food} (Confidence: {conf:.2f})")

Input: 'I want something cold and sweet with chocolate sauce' → Predicted: ice_cream (Confidence: 0.98)
Input: 'A cold dessert that melts in the mouth would be nice' → Predicted: ice_cream (Confidence: 0.97)
Input: 'Something creamy and chilled for a hot day' → Predicted: ice_cream (Confidence: 0.29)
Input: 'I’d love a sweet treat that comes in a cone or cup' → Predicted: ice_cream (Confidence: 0.99)
Input: 'Maybe something frozen with vanilla and strawberry flavors' → Predicted: ice_cream (Confidence: 0.48)
Input: 'A dessert that cools me down instantly' → Predicted: apple_pie (Confidence: 0.45)
Input: 'I’m thinking of something that melts if you don’t eat it fast' → Predicted: ice_cream (Confidence: 0.99)
Input: 'A cold snack that makes me feel refreshed' → Predicted: ice_cream (Confidence: 0.75)
Input: 'Something icy and smooth after dinner' → Predicted: ice_cream (Confidence: 0.80)
Input: 'Something sweet and cold, maybe with sprinkles' → Predicted: pizza (Confidence: 0.44)
Input: 

ในส่วนของขั้นตอนนี้ทำหน้าที่  
- สร้างฟังก์ชัน `predict_food()` ที่รับข้อความเข้ามา แล้วผ่านกระบวนการเดียวกับชุดฝึก: ทำความสะอาดข้อความ → แปลงเป็นลำดับตัวเลข → ปรับความยาว → ทำนายด้วยโมเดล  
- ใช้ `argmax` เพื่อหาดัชนีของหมวดอาหารที่มีคะแนนความน่าจะเป็นสูงสุด และแปลงกลับเป็นชื่ออาหารด้วย `label_encoder.inverse_transform()`  
- คำนวณระดับความมั่นใจ (confidence) จากค่าความน่าจะเป็นสูงสุดของการทำนาย  
- ทดสอบฟังก์ชันกับชุดข้อความตัวอย่างที่เขียนในรูปแบบโดยนัย (ไม่เอ่ยชื่ออาหารตรงๆ) เพื่อประเมินว่าโมเดลเข้าใจบริบทได้ดีเพียงใด  
- แสดงผลลัพธ์การทำนายพร้อมระดับความมั่นใจ เพื่อให้เห็นภาพชัดเจนว่าโมเดลสามารถใช้งานจริงในระบบแนะนำเมนูได้หรือไม่